# Wrangling WeRateDogs twitter archive data

This is a wrangling report about the WeRateDogs twitter archive project. In this projects we used the folowing libraries:
- pandas (To manage dataframes)
- numpy (To use special mathematical functions)  
- json (To manage json objects and files)
- tweepy (To access the twitter API)
- requests (To download files through HTTP)
- time (To calculate duration of certain operations)
- matplotlib (To plot charts)
- re (To use regex format)

***

### Table of content
<ol>
    <li><a href="#gathering">Gathering</a></li>
    <li><a href="#assessing">Assessing</a></li>
    <li><a href="#cleaning">Cleaning</a></li>
</ol>

***

<a id="gathering"></a>
## Gathering


As the first step of the wrangling procedure, we begun by collecting the data that we wanted to use in this project. We have used three different methods in this step:
1. Downloading manually a .csv file to the workspace.
2. Downloading a .csv file programatically to the workspace.
3. Using Twitter API to gather additional data ('retweet_count','favorite_count') from WeRateDogs archive.

After collecting the data, we had three .csv files that we're going to assess and clean.
The files are:
- twitter-archive-enhanced.csv: Contains 2356, lines where each line contains some informations about 1 tweet.
- image-predections.tsv: Contains 2075 lines, where each line presents the results of a NN used to classify the image in the corresponding tweet. 
- Additional data: 
    - twitter_additional_data.csv: Contains addtional data to the tweets present in twitter-archive-enhanced.csv
    - not_found_tweets_id_list.txt: List of the tweets ids present in twitter-archive-enhanced.csv that are deleted from twitter.

<a id="assessing"></a>
## Assessing

Once we have gathered the data we need, we can advance to the next step which is assessing the collected data.
The assessing process we implemented is the following:
1. Checking the dataframes we have using the basic methods to see if we can spot any quality issues like, missing values from columns, non-accurate data types, non-descriptive column names,..etc. Or any tidiness issues like, variables being used as columns, columns in the wrong dataframe, ..etc.
    - The notable used methods are: `.head()`, `.info()`, `.len()`.
    
After the visual and quick assessment of the dataframes we have, we decided to focused on investigating in details the twitter-archive-enhanced dataframe because it presents the core of the data we have (Tweets), and it has a lot of quality and tidiness issues.
2. Investigating and determining the 3 categories (tweets, replies, retweets) of the tweets we have:
    - Since a tweet can have multiple replies, we invisticated whether we have more than one reply to a certain tweet, and we found only one tweet have 2 replies. Precisely, the second reply contains an upgreade of the ratings assignes to the dog in the tweet. Then we made sure that those replies are not among the delted tweets else they would be both dropped in the cleaning phase.
    - Because we want to use only original tweets than we won't be using any retweets. However, when collecting the additional data through twitter API, we identified only 163 retweets instead 181 which are present in the twitter-archive-enhanced. After investigating this problem we found out that this issue was caused by the 18 retweets that were deleted from twitter, thus eliminating the possibility of mis-extraction of data from twitter.
    - The notable used methods are: `.notnull()`, `.shape[]`, `.duplicated()`, `.query()`. `.sort_values()`.
    
3. Assessing of the columns 'source' using the `.value_counts()` method made the issue regarding the qality of this column visible. The source should states one of the following values 'Twitter for iPhone', 'Vine - Make a Scene', 'Twitter Web Client', or 'TweetDeck'.

4. Investigation of the ratings columns:
    - Many ratings are wrongly extracted from the tweet text.
    - Most common used rating denominator is 10 and all the denominators that are below 10 are wrongly extracted from the tweet texts.
    - Most common used rating numerator are in the range [4,12]. Some of the ratings with numerator bigger than 10 are just a multiplied form of a normal rating form (xx/10).
    - Many dogs have names like 'a' and 'an' which are obviously wrongly extracted from the tweets.
There are many more Quality issues regarding the data we have, however we have limited our invistigation to extract an accepted number of quality and tidiness issues.

<a id="cleaning"></a>
## Cleaning

After assessment of the present data, we can now pass to the cleaning phase.
The cleaning process is as follows:
- Address every quality or tidiness issue on 3 steps:
    1. Define: We define the action we're going to take to solve the issue
    2. Code: Treat the issue as described in the define step.
    3. Test: Check if the code we implemented did the cleaning job as should be.
    
We have applied this process to each issue we noted above, however it's important to order the cleaning tasks we have:
Example: Dropping the records and columns un-wanted is a task of high priority, since celaning something that would be deleted afterwards make us waste time and requires more computational effort.

- The retweets and the deleted tweets have been dropped from the data we have.
- The ratings have been re-exracted from the tweets texts and the tweets with rating numerators less than 5 were deleted.
- The rows with rating denominator less than 10 were dropped.
- The column rating_denominator have been dropped because now all the rows have rating denominator equal 10.
- The multiplied ratings were reduced to the form 'xx/10'
- The dogs named 'a' and 'an' were reamed 'None'
- The stage comlumns are dropped and replaced by one column named 'stage' which take values from, 'doggo','pupper', 'puppo' or 'floofer'.
- Only the information relatied to the highest prediction confidence are kept. The second and third prediction based on confidence coeficient were dropped.
- Two master table created, linked via a column 'tweet_id':
    1. `twitter_archive_master.csv` which contains only the data regarding the tweet.
    2. `dogs_master.csv` which contains all the information about the dogs.